In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})

In [ ]:
import plumed
import wham

### Equilibrium run

In [ ]:
colvar=plumed.read_as_pandas("colvar.dat")

In [ ]:
fig, ax = plt.subplots()
ax.plot(colvar.time,colvar.phi,".",label="$\phi$")
ax.plot(colvar.time,colvar.psi,".",label="$\psi$")

ax.set_ylim(-np.pi, np.pi)
ax.set_xlim(colvar.time[0], np.max(colvar.time))
ax.set_xlabel("time")
ax.set_ylabel("Torsion angle")

plt.legend()
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(colvar.phi,colvar.psi,"o", ms=0.5)
ax.set_xlabel("$\phi$")
ax.set_ylabel("$\psi$")
ax.set_xlim((-np.pi,np.pi))
ax.set_ylim((-np.pi,np.pi))
plt.show()

In [ ]:
fes_phi = plumed.read_as_pandas("fes_phi.dat").replace([np.inf, -np.inf], \
                                                     np.nan).dropna()
fes_psi = plumed.read_as_pandas("fes_psi.dat").replace([np.inf, -np.inf], \
                                                     np.nan).dropna()

In [ ]:
fig, ax = plt.subplots()
ax.plot(fes_phi.phi,fes_phi.ffphi,label="$\phi$")
ax.plot(fes_psi.psi,fes_psi.ffpsi,label="$\psi$")
ax.set_xlim((-np.pi,np.pi))
ax.set_ylim((-2,10))
ax.set_xlabel("Torsion angle")
ax.set_ylabel("Free Energy")
plt.legend()
plt.show()

In [ ]:
kBT=300*8.314462618*0.001 # use kJ/mol here

col=[]
for i in range(32):
    col.append(plumed.read_as_pandas("colvar_multi_" + str(i)+".dat"))

bias = np.zeros((len(col[0]["bb.bias"]),32))
for i in range(32):
    bias[:,i] = col[i]["bb.bias"][-len(bias):]
w = wham.wham(bias,T=kBT)
colvar = col[0]
colvar["logweights"] = w["logW"]
plumed.write_pandas(colvar,"bias_multi.dat")

In [ ]:
col=[]
for i in range(32):
    col.append(plumed.read_as_pandas("colvar_multi_" + str(i)+".dat"))

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
for i in range(32):
    ax.plot(col[i].phi[2001*i:2001*(i+1)],col[i].psi[2001*i:2001*(i+1)], ".")
ax.set_xlabel("$\phi$")
ax.set_ylabel("$\psi$")
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
plt.show()
# in this graph you can appreciate which region was sampled by each simulation


In [ ]:
fig, ax = plt.subplots()
fes_phi=plumed.read_as_pandas("fes_phi.dat").replace([np.inf, -np.inf], np.nan).dropna()
ax.plot(fes_phi.phi,fes_phi.ffphi,label="original")
fes_phib=plumed.read_as_pandas("fes_phi_cat.dat").replace([np.inf, -np.inf], np.nan).dropna()
ax.plot(fes_phib.phi,fes_phib.ffphi,label="biased")
fes_phir=plumed.read_as_pandas("fes_phi_catr.dat").replace([np.inf, -np.inf], np.nan).dropna()
ax.plot(fes_phir.phi,fes_phir.ffphir,label="reweighted")
plt.legend()
ax.set_xlim((-np.pi,np.pi))
ax.set_xlabel("$\phi$")
ax.set_ylabel("$F(\phi)$")
plt.show()
